In [1]:
import os
from pathlib import Path
import time

from arcgis import GIS, GeoAccessor
from dotenv import find_dotenv, load_dotenv
from modeling import Country, ModelingAccessor

load_dotenv(find_dotenv())

# load the "autoreload" extension so as src code is changed, the changes are picked up in the dataframe
%load_ext autoreload
%autoreload 2

ImportError: cannot import name 'GIS' from 'arcgis' (/Users/joel5174/miniconda3/envs/arcgis/lib/python3.7/site-packages/arcgis/__init__.py)

In [ ]:
project_parent = Path('.').absolute().parent.parent
dir_data = project_parent/'data'
dir_int = dir_data/'interim'
dir_raw = dir_data/'raw'
gdb_int = dir_int/'interim.gdb'
gdb_raw = dir_raw/'raw.gdb'

biz_drop_cols = ['OBJECTID', 'CONAME','SALESVOL', 'HDBRCH', 'ULTNUM', 'PUBPRV', 'EMPNUM', 'FRNCOD', 'ISCODE', 'SQFTCODE', 'LOC_NAME', 'STATUS', 'SCORE', 'SOURCE', 'REC_TYPE']

In [ ]:
# gis = GIS(os.getenv('ESRI_PORTAL_URL'), username=os.getenv('ESRI_PORTAL_USERNAME'), password=os.getenv('ESRI_PORTAL_PASSWORD'))
# gis = GIS(os.getenv('BASRV_QA_URL'), username=os.getenv('BASRV_QA_USERNAME'), password=os.getenv('BASRV_QA_PASSWORD'))
gis = GIS(os.getenv('BA_QA_URL'), username=os.getenv('BA_QA_USERNAME'), password=os.getenv('BA_QA_PASSWORD'))
usa = Country('USA')

usa

In [ ]:
%%time
aoi_df = usa.cbsas.get('seattle')

aoi_df

In [ ]:
evars = usa.enrich_variables

print(evars.info())

key_vars = evars[
    (evars.name.str.endswith('CY'))
    & (evars.data_collection.str.lower().str.contains('key'))
].reset_index(drop=True)

print(key_vars.info())
key_vars

In [ ]:
%%time
bg_df = aoi_df.mdl.level(0).get().mdl.enrich(key_vars)

print(bg_df.info())
bg_df.head()

In [ ]:
%%time
biz_df = aoi_df.mdl.business.get_by_name('ace hardware')

print(biz_df.info())
biz_df.head()

In [ ]:
%%time
bg_near_biz_df = bg_df.mdl.proximity.get_nearest(biz_df, origin_id_column='ID', destination_count=3, near_prefix='brand')

print(bg_near_biz_df.info())
bg_near_biz_df.head()

In [ ]:
%%time
comp_df = aoi_df.mdl.business.get_competition(biz_df, local_threshold=3)
comp_df.spatial.set_geometry('SHAPE')

print(comp_df.info())
comp_df.head()

In [ ]:
%%time
bg_near_comp_df = bg_df.mdl.proximity.get_nearest(comp_df, origin_id_column='ID', near_prefix='comp', destination_count=6,
                                                  destination_columns_to_keep=['brand_name', 'brand_name_category'])

print(bg_near_comp_df.info())
bg_near_comp_df.head()

In [ ]:
%%time
biz_near_comp_df = biz_df.mdl.proximity.get_nearest(comp_df, origin_id_column='LOCNUM', near_prefix='biz_comp', 
                                                    destination_id_column='LOCNUM', destination_count=6,
                                                    destination_columns_to_keep=['brand_name', 'brand_name_category'])

print(biz_near_comp_df.info())
biz_near_comp_df.head()